<a href="https://colab.research.google.com/github/TwoGunTitan/Vision-without-glasses/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
# from google.colab import files
import os
# import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
def generate_file_names(start, end, directory):
    classes = ['cat.', 'dog.']
    file_names = []
    ## 0 is cat and 1 is dog
    for i in range(start, end):
        for label in range(2):
            file_names.append([directory + classes[label] + str(i) + '.jpg', label])
    return file_names

def load_images(start,end, directory):
    file_names = generate_file_names(start,end, directory)
    np.random.shuffle(file_names)
    dataset = []
    images = []
    labels = []

    i=0

    for name, label in file_names:
        img = image.load_img(name, target_size=(224, 224))
        x = image.img_to_array(img)
        x = tf.keras.applications.inception_v3.preprocess_input(x)
        images.append(x)
        labels.append(label)
        print(i)
        i = i+1

    return np.array(images), np.array(labels)

In [ ]:
base_model = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)
# print('debug')
base_model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D()(x)

outputs = tf.keras.activations.sigmoid(keras.layers.Dense(1)(x))

model = keras.Model(inputs, outputs)


x, y = load_images(2500, 5000, '/content/drive/MyDrive/dogs_vs_cats_dataset/train_data/cyblur_4/')
# print('debug')
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.summary()

In [ ]:
model.fit(x=x,y=y, epochs=10)
model.save_weights('Resnet_gaussian_4')